# **Chapter 1: Introduction to CI/CD**

Welcome to the journey of mastering modern software delivery. Before we dive into Docker containers, Kubernetes orchestration, or pipeline configurations, we must first understand the foundational philosophy that ties everything together: Continuous Integration and Continuous Deployment (CI/CD). This chapter establishes the conceptual bedrock upon which all subsequent technical implementations rest. Whether you are a developer looking to streamline your workflow, an operations engineer aiming to reduce deployment anxiety, or a team lead seeking to accelerate delivery without compromising quality, understanding CI/CD principles is non-negotiable in today's cloud-native landscape.

---

### 1.1 What is Continuous Integration?

**Continuous Integration (CI)** is a software development practice where developers frequently merge their code changes into a central repository, followed by automated builds and tests. The core goal is to detect integration errors as quickly as possible, which is achieved by integrating at least once per day, though modern teams often integrate multiple times per day.

#### The Core Mechanics

Imagine a team of five developers working on separate features of an application. Without CI, each developer works in isolation for days or weeks, then attempts to merge their changes together. This "integration hell" results in merge conflicts, broken builds, and delayed releases. CI solves this by enforcing a simple rule: **the main branch must always be in a deployable state**.

Here is what happens in a typical CI workflow:

1. **Code Commit**: Developer A completes a feature and commits code to a feature branch
2. **Pull Request**: Developer A creates a pull request (PR) to merge into the main branch
3. **Automated Trigger**: The CI system detects the PR and triggers a pipeline
4. **Build Phase**: The application is compiled/built in a clean environment
5. **Test Phase**: Unit tests, integration tests, and linting checks run automatically
6. **Report**: Results are reported back to the PR—green (pass) or red (fail)
7. **Merge**: Only if green, the code is merged into the main branch

#### Industry Standards for CI

Modern CI adheres to these principles established by Martin Fowler and the Extreme Programming community, refined over two decades:

- **Maintain a Single Source Repository**: All code lives in a version control system (Git is the industry standard). Everything needed to build the project should be in the repository—code, tests, configuration files, and documentation.

- **Automate the Build**: The build process should be automated to the point where a single command or trigger can execute it. No manual steps should be required to create a deployable artifact.

- **Make Your Build Self-Testing**: Automated tests should validate that the build works correctly. Industry standard mandates at minimum unit tests, but ideally includes integration tests and static analysis.

- **Every Commit Should Build on an Integration Machine**: Developers' local environments may have hidden dependencies or configurations. Building on a clean, controlled CI server ensures the build is truly portable.

- **Keep the Build Fast**: If a build takes too long, developers will avoid committing frequently. The industry benchmark is under 10 minutes for the commit stage, though complex systems may have longer pipelines with parallel stages.

- **Test in a Clone of the Production Environment**: Use containers (which we will cover extensively) to ensure that "it works on my machine" translates to "it works in production."

- **Make it Easy to Get the Latest Deliverables**: Anyone should be able to download the latest successful build artifacts without needing to build from source.

#### Practical Example: A CI Workflow

Consider a Node.js application. A proper CI setup would look like this:

```yaml
# Conceptual CI Configuration (GitHub Actions style)
name: Continuous Integration

on:
  pull_request:
    branches: [ main ]
  push:
    branches: [ main ]

jobs:
  build-and-test:
    runs-on: ubuntu-latest
    
    steps:
      # 1. Checkout code
      - uses: actions/checkout@v4
      
      # 2. Setup environment (clean, reproducible)
      - name: Setup Node.js
        uses: actions/setup-node@v4
        with:
          node-version: '20'
          cache: 'npm'
      
      # 3. Install dependencies
      - name: Install dependencies
        run: npm ci
      
      # 4. Run linting (code quality)
      - name: Lint code
        run: npm run lint
      
      # 5. Run unit tests
      - name: Run tests
        run: npm test
      
      # 6. Build application
      - name: Build
        run: npm run build
      
      # 7. Upload build artifacts
      - name: Upload artifacts
        uses: actions/upload-artifact@v4
        with:
          name: build-files
          path: ./dist
```

**Key Takeaway**: CI is not just about running tests; it is about creating a **fast feedback loop** that validates code quality and integration success before human code review even begins.

---

### 1.2 What is Continuous Deployment?

**Continuous Deployment (CD)** extends Continuous Integration by automatically deploying every change that passes the automated test suite to production. This is the most advanced form of CI/CD, where code commits can go live within minutes, assuming all quality gates are met.

#### Understanding the Deployment Spectrum

It is crucial to distinguish between **Continuous Delivery** (covered next) and **Continuous Deployment**, as they are often conflated:

- **Continuous Delivery**: Code is automatically built and tested, and *can* be deployed to production at any time, but requires a human decision (clicking a button) to actually deploy.
- **Continuous Deployment**: Code is automatically built, tested, *and* deployed to production without human intervention.

Continuous Deployment represents the pinnacle of automation trust. Companies like Netflix, Amazon, and Etsy deploy hundreds or thousands of times per day using this model.

#### The Deployment Pipeline

A Continuous Deployment pipeline is essentially a production line with quality checkpoints:

```
Code Commit → Build → Unit Tests → Integration Tests → Security Scan → Staging Deploy → E2E Tests → Production Deploy → Verification
```

Each stage acts as a filter. If any stage fails, the pipeline stops, and the code does not progress further. Only code that survives every automated gate reaches production.

#### Industry Standards for Continuous Deployment

- **Comprehensive Test Coverage**: You cannot safely deploy automatically without high confidence in your test suite. Industry standard requires >80% code coverage for unit tests, plus robust integration and contract tests.

- **Feature Flags**: Since deploying unfinished features is inevitable when deploying frequently, use feature flags (toggles) to hide incomplete functionality from users. Tools like LaunchDarkly, Unleash, or simple environment variables enable this.

- **Canary Releases**: Rather than deploying to 100% of users immediately, deploy to a small percentage (5-10%), monitor metrics, then gradually roll out. Kubernetes makes this straightforward with rolling updates and service meshes.

- **Automated Rollbacks**: If error rates spike or latency increases after a deployment, the system should automatically roll back to the previous version. This requires sophisticated monitoring (discussed in Part VIII).

- **Immutable Infrastructure**: Never modify running servers. Instead, create new instances with the new code and replace old ones. This ensures consistency and easy rollback.

#### Practical Example: Deployment Automation

Here is how a deployment step might look in a pipeline:

```yaml
# Conceptual deployment stage
deploy-to-production:
  needs: [integration-tests, security-scan]
  runs-on: ubuntu-latest
  
  steps:
    - name: Configure kubectl
      run: |
        echo "${{ secrets.KUBECONFIG }}" | base64 -d > kubeconfig
        export KUBECONFIG=kubeconfig
    
    - name: Deploy to Kubernetes
      run: |
        # Apply new version using kubectl
        kubectl set image deployment/app \
          app=myregistry/app:${{ github.sha }} \
          --record
        
        # Wait for rollout to complete
        kubectl rollout status deployment/app
        
        # Verify health
        kubectl get pods -l app=frontend
```

**Key Takeaway**: Continuous Deployment requires **organizational maturity**. It is not merely a technical implementation but a cultural commitment to quality, monitoring, and rapid recovery.

---

### 1.3 What is Continuous Delivery?

**Continuous Delivery** is the practice of keeping your codebase in a state where it can be released to production at any time. It is the safety-conscious sibling of Continuous Deployment—the automation is identical, but the final deployment decision includes a human approval step.

#### Why Choose Continuous Delivery?

Not every organization can (or should) adopt full Continuous Deployment. Regulatory requirements, complex coordination needs, or business processes may require human oversight. Continuous Delivery provides 90% of the benefits:

1. **Reduced Deployment Risk**: Smaller, more frequent changes are less risky than massive quarterly releases.
2. **Immediate Value Realization**: Features are ready to ship the moment business decides they want them live.
3. **Rapid Feedback**: Stakeholders can see and test features in production-like environments immediately.
4. **Deployment Confidence**: Since the deployment process is automated and tested hundreds of times in staging, the production deployment is routine, not terrifying.

#### The Delivery Pipeline Architecture

The Continuous Delivery pipeline extends the CI pipeline with deployment stages:

```
Source → Build → Test → Staging → [Manual Gate] → Production
```

The manual gate can be:
- A button click in a CI/CD dashboard
- A Slack approval
- A scheduled time window (deploy only during business hours)
- A business approval workflow

#### Industry Best Practices

- **Environment Parity**: Staging must mirror production as closely as possible. Use the same infrastructure (Kubernetes), same data stores (with anonymized data), and same network configurations.

- **Deployment Automation**: Even with a manual gate, the actual deployment should be one-click or automated upon approval. No manual SSHing into servers to copy files.

- **Configuration Management**: Separate environment-specific configuration from code. Use environment variables, ConfigMaps (Kubernetes), or external configuration services. Never hardcode database URLs or API keys.

- **Database Migration Strategy**: Schema changes must be backward compatible. Deploy code first, then run migrations, or use the expand/contract pattern (add new column → dual-write → migrate data → switch reads → remove old column).

#### Practical Example: Blue/Green Deployment

A common Continuous Delivery pattern is Blue/Green deployment, where you maintain two identical production environments:

```bash
# Conceptual Blue/Green deployment script
# Current production is "blue", we deploy to "green"

# 1. Deploy new version to green environment
kubectl apply -f k8s/green-deployment.yaml

# 2. Run smoke tests against green
curl -f http://green-environment/health

# 3. Switch traffic from blue to green
kubectl patch service production-service -p \
  '{"spec":{"selector":{"version":"green"}}}'

# 4. Keep blue running for quick rollback if needed
# If issues arise, switch back to blue immediately
```

**Key Takeaway**: Continuous Delivery is about **preparedness**. The code is always production-ready, even if business timing delays the actual release.

---

### 1.4 The CI/CD Pipeline Architecture

A **pipeline** is the orchestrated sequence of stages that code moves through from commit to production. Understanding pipeline architecture is essential for designing efficient, maintainable workflows.

#### Pipeline Components

**1. Source Stage**
- Triggered by code commits, pull requests, or scheduled timers
- Checks out code from repository
- Performs initial validation (file syntax, commit message format)

**2. Build Stage**
- Compiles source code into artifacts
- Creates Docker images
- Tags artifacts with version identifiers (typically Git commit SHA)

**3. Test Stage**
- **Static Analysis**: Linting, code formatting, security scanning (SAST)
- **Unit Tests**: Fast, isolated tests of individual functions/components
- **Integration Tests**: Testing component interactions, often with test databases
- **Contract Tests**: Verifying API compatibility between services

**4. Artifact Stage**
- Pushing Docker images to registries
- Storing build artifacts
- Signing artifacts for provenance

**5. Deployment Stages**
- **Development**: Auto-deploy for immediate developer testing
- **Staging**: Pre-production environment for QA and stakeholder review
- **Production**: Live environment serving real users

**6. Verification Stage**
- Smoke tests (is the app running?)
- Health checks
- Synthetic monitoring (fake transactions to test functionality)

#### Pipeline Patterns

**Fan-Out/Fan-In**: Run independent stages in parallel (fan-out), then wait for all to complete before proceeding (fan-in).

```
Build → [Unit Tests + Lint + Security Scan] → Integration Tests
```

**Deployment Rings**: Progressive exposure:
1. Ring 0: Internal users only
2. Ring 1: Beta customers
3. Ring 2: Specific regions
4. Ring 3: Global availability

**Pipeline as Code**: Modern CI/CD defines pipelines in code (YAML files) stored in the repository, versioned alongside the application code. This enables:
- Code review of pipeline changes
- History and audit trails
- Reusability through templates

#### Example: Complete Pipeline Architecture

```yaml
# Conceptual complete pipeline
stages:
  - build
  - test
  - security
  - deploy-staging
  - e2e-tests
  - deploy-production

variables:
  DOCKER_IMAGE: $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA

build:
  stage: build
  script:
    - docker build -t $DOCKER_IMAGE .
    - docker push $DOCKER_IMAGE

unit-tests:
  stage: test
  needs: [build]
  parallel:
    matrix:
      - TEST_SUITE: [auth, payment, inventory]
  script:
    - npm run test:$TEST_SUITE

security-scan:
  stage: security
  needs: [build]
  script:
    - trivy image $DOCKER_IMAGE

deploy-staging:
  stage: deploy-staging
  environment: staging
  script:
    - helm upgrade --install app ./chart --set image.tag=$CI_COMMIT_SHA

e2e-tests:
  stage: e2e-tests
  needs: [deploy-staging]
  script:
    - npx cypress run

deploy-production:
  stage: deploy-production
  environment: production
  when: manual  # This makes it Continuous Delivery, not Deployment
  only:
    - main
  script:
    - helm upgrade --install app ./chart --set image.tag=$CI_COMMIT_SHA
```

**Key Takeaway**: Treat your pipeline as a **critical application**. It requires testing, versioning, and maintenance just like production code.

---

### 1.5 Benefits of CI/CD

Organizations adopting CI/CD correctly experience transformative improvements across technical, business, and cultural dimensions.

#### Technical Benefits

**Reduced Integration Risk**
When developers integrate daily, conflicts are caught immediately while the context is fresh. The cost of fixing integration issues increases exponentially with time. CI reduces "merge hell" and broken main branches.

**Faster Feedback Loops**
Developers know within minutes if their changes broke the build or failed tests. This immediate feedback enables rapid iteration and debugging while the code context is still in short-term memory.

**Automated Quality Assurance**
Machines execute tests consistently; humans do not. Automated pipelines ensure that every code change undergoes identical validation, eliminating "it worked on my machine" variability.

**Infrastructure Consistency**
Infrastructure as Code (IaC) and containerization ensure that development, staging, and production environments are identical, eliminating environment-specific bugs.

#### Business Benefits

**Accelerated Time-to-Market**
Features move from concept to customer in hours or days rather than months. This speed enables competitive advantage and rapid response to market changes.

**Reduced Cost of Change**
Small, frequent changes are cheaper to develop and fix than large releases. When a bug is introduced, it is immediately obvious which small change caused it, reducing mean time to recovery (MTTR).

**Improved Resource Allocation**
Automating repetitive deployment tasks frees engineers to focus on feature development and innovation rather than manual release coordination.

**Enhanced Customer Satisfaction**
Frequent updates mean customers receive bug fixes and features faster. Continuous feedback loops enable rapid response to customer needs.

#### Cultural Benefits

**Psychological Safety**
When deployments are routine and low-risk, teams are not afraid to deploy on Fridays or during critical periods. This reduces "deployment anxiety" and burnout.

**Collaboration**
CI/CD breaks down silos between development and operations. Shared responsibility for the pipeline fosters empathy and cross-functional understanding.

**Transparency**
The pipeline provides visibility into the health of the codebase and the progress of features. Stakeholders can see exactly what is deployed and what is pending.

#### Measurable Metrics (DORA)

The DevOps Research and Assessment (DORA) team identified four key metrics improved by CI/CD:

1. **Deployment Frequency**: How often deployments occur (elite performers deploy on-demand, multiple times per day)
2. **Lead Time for Changes**: Time from commit to production (elite: less than one hour)
3. **Change Failure Rate**: Percentage of deployments causing production failures (elite: 0-15%)
4. **Time to Restore Service**: Time to recover from failure (elite: less than one hour)

**Key Takeaway**: CI/CD is not just about tools; it is about **business agility**. Organizations mastering these practices deploy 208 times more frequently and recover from failures 2,604 times faster than low performers (DORA State of DevOps Report).

---

### 1.6 Common CI/CD Challenges

While the benefits are substantial, implementing CI/CD is not without obstacles. Awareness of these challenges enables proactive mitigation.

#### Technical Challenges

**Flaky Tests**
Non-deterministic tests that pass sometimes and fail others erode trust in the pipeline. Teams begin ignoring failures, leading to real bugs slipping through.
- *Solution*: Quarantine flaky tests immediately, investigate root causes (usually timing issues or external dependencies), and require tests to be deterministic.

**Long-Running Pipelines**
Pipelines taking hours discourage frequent commits and slow feedback loops.
- *Solution*: Parallelize independent jobs, optimize test suites (remove redundant tests), use incremental builds, and leverage caching aggressively.

**Environment Drift**
When staging differs from production (different data volumes, missing services), deployments that passed testing fail in production.
- *Solution*: Use containers to ensure consistency, implement infrastructure as code, and regularly verify environment parity.

**Secret Management**
Hardcoding secrets in pipelines or repositories creates security vulnerabilities.
- *Solution*: Use dedicated secret management tools (HashiCorp Vault, AWS Secrets Manager, Kubernetes Secrets), rotate credentials regularly, and never log sensitive data.

#### Organizational Challenges

**Cultural Resistance**
Teams accustomed to quarterly releases may resist daily deployments due to fear or comfort with existing processes.
- *Solution*: Start with Continuous Delivery (manual approval) before Continuous Deployment, demonstrate small wins, and provide training.

**Siloed Teams**
When Dev and Ops report to different managers with conflicting KPIs (features vs. stability), CI/CD adoption stalls.
- *Solution*: Align incentives around shared goals (deployment frequency, uptime), create cross-functional teams, and implement DevOps practices.

**Legacy Codebases**
Monolithic applications with tight coupling and no test coverage are difficult to integrate into modern pipelines.
- *Solution*: Implement the Strangler Fig pattern (gradually replace components), prioritize writing tests for modified code (boy scout rule), and consider breaking the monolith into services.

**Compliance and Audit Requirements**
Regulated industries (finance, healthcare) require documentation and approval gates that seem to conflict with automation.
- *Solution*: Implement audit trails in pipelines (who deployed what, when), use automated compliance checks, and maintain electronic signatures for approvals.

#### Pipeline Anti-Patterns to Avoid

1. **The "Big Bang" Integration**: Waiting until the end of a sprint to integrate all features. This defeats the purpose of CI.
2. **Production Data in Tests**: Using production databases for testing risks data corruption and violates privacy regulations.
3. **Ignoring Broken Builds**: Allowing the main branch to remain broken for days creates a cascade of issues for other developers.
4. **Manual Deployments**: Even with CI, if deployment involves manual file copying or configuration editing, the process is error-prone.
5. **Snowflake Servers**: Manually configured servers that cannot be reproduced automatically. If you cannot rebuild your server in 15 minutes, you have a problem.

**Key Takeaway**: CI/CD is a **journey, not a destination**. Start with basic automation, measure results, and iteratively improve. Perfection is not required on day one, but commitment to improvement is.

---

### 1.7 Industry Standards and Best Practices

To conclude this foundational chapter, let us establish the non-negotiable standards that separate amateur CI/CD implementations from professional-grade systems.

#### The Twelve-Factor App Methodology

While not exclusively about CI/CD, the Twelve-Factor App provides the architectural foundation for successful pipelines:

1. **Codebase**: One codebase tracked in revision control, many deploys
2. **Dependencies**: Explicitly declare and isolate dependencies (use lock files)
3. **Config**: Store config in environment variables, never in code
4. **Backing Services**: Treat databases, queues, and caches as attached resources
5. **Build, Release, Run**: Strictly separate build and run stages
6. **Processes**: Execute the app as one or more stateless processes
7. **Port Binding**: Export services via port binding
8. **Concurrency**: Scale out via the process model
9. **Disposability**: Fast startup and graceful shutdown enable robust CI/CD
10. **Dev/Prod Parity**: Keep development, staging, and production as similar as possible
11. **Logs**: Treat logs as event streams (don't write to files in containers)
12. **Admin Processes**: Run admin/management tasks as one-off processes

#### CI/CD Best Practices Checklist

**Repository Management**
- [ ] Main branch is always deployable
- [ ] Branch protection rules prevent direct pushes to main
- [ ] All changes via Pull Request with at least one reviewer
- [ ] Commit messages follow conventional commit standards
- [ ] Repository includes README, LICENSE, and CONTRIBUTING files

**Pipeline Design**
- [ ] Pipeline configuration stored as code (YAML) in repository
- [ ] Stages are idempotent (running twice produces same result)
- [ ] Failed pipelines block deployment
- [ ] Secrets injected via environment variables, never hardcoded
- [ ] Artifacts versioned with Git commit SHA

**Testing Standards**
- [ ] Unit tests run in under 5 minutes
- [ ] Code coverage reported on every PR
- [ ] Integration tests use test containers or ephemeral environments
- [ ] Security scanning (SAST/SCA) mandatory before deployment
- [ ] Performance regression tests for critical paths

**Deployment Standards**
- [ ] Blue/Green or Canary deployment strategies implemented
- [ ] Database migrations are backward compatible
- [ ] Feature flags control new functionality
- [ ] Automated rollback procedures tested regularly
- [ ] All deployments logged with who, what, when

**Security Standards**
- [ ] No secrets in Git history (use git-secrets or similar)
- [ ] Container images scanned for vulnerabilities
- [ ] Least privilege access to CI/CD systems
- [ ] Signed artifacts for supply chain verification
- [ ] Regular rotation of credentials and tokens

#### Industry Frameworks and Specifications

**SLSA (Supply-chain Levels for Software Artifacts)**
A security framework from Google and the OpenSSF ensuring software integrity:
- Level 1: Basic automation (build scripts)
- Level 2: Signed provenance (know who built what)
- Level 3: Hardened builds (isolated, ephemeral, parameterless)
- Level 4: Reproducible builds (bit-for-bit identical)

**OpenTelemetry**
Standard for observability (metrics, logs, traces) in CI/CD pipelines, ensuring you can debug pipeline failures as effectively as application failures.

**OCI (Open Container Initiative)**
Standards for container formats and runtimes, ensuring your Docker images work across any compliant platform.

**Key Takeaway**: Following industry standards transforms CI/CD from a custom script collection into a **reliable, maintainable, and secure engineering system** that can scale with your organization.

---

### 1.8 Quick History of DevOps Evolution

Understanding where we came from illuminates why current practices exist.

#### The Pre-DevOps Era (Pre-2001)
Software was released in "waterfall" cycles of 6-18 months. Development wrote code and "threw it over the wall" to Operations, who struggled to deploy unfamiliar software. The result: "It works on my machine" conflicts, delayed releases, and failed deployments.

#### The Agile Revolution (2001-2009)
The Agile Manifesto (2001) emphasized working software over documentation and responding to change over following a plan. This accelerated development but created a bottleneck at deployment. Operations became the constraint.

#### Continuous Integration Emerges (1990s-2010s)
Kent Beck introduced CI as part of Extreme Programming (XP) in the late 1990s. CruiseControl (2001) and Jenkins (originally Hudson, 2004) automated builds. The practice focused on integrating code frequently, but deployment remained manual.

#### The DevOps Movement (2009-Present)
Patrick Debois coined "DevOps" in 2009, recognizing that Development and Operations must collaborate. The goal: break down silos, automate the entire software lifecycle, and treat infrastructure as code.

#### Cloud Native and Containers (2013-Present)
Docker (2013) revolutionized consistency between environments. Kubernetes (2014) standardized orchestration. CI/CD became cloud-native, with pipelines creating immutable container images rather than configuring servers.

#### GitOps and Platform Engineering (2017-Present)
Modern evolution treats Git as the single source of truth for both application code and infrastructure. Platform Engineering teams create internal developer platforms, abstracting CI/CD complexity so developers can focus on features.

#### Current Trends (2024-Present)
- **AI-Augmented CI/CD**: Using machine learning to predict flaky tests, optimize build times, and auto-remediate failures
- **Supply Chain Security**: SBOMs (Software Bill of Materials) and signed artifacts becoming mandatory
- **Platform Engineering**: Self-service platforms reducing cognitive load on developers
- **Internal Developer Portals**: Backstage and similar tools providing unified CI/CD visibility

---

### Chapter Summary and Preview

In this chapter, we established the foundational concepts of CI/CD. You learned that **Continuous Integration** ensures code integrates cleanly through automated builds and tests, **Continuous Delivery** keeps code in a deployable state with manual approval for production, and **Continuous Deployment** automates the entire path to production. We explored the architecture of pipelines, the substantial benefits across technical, business, and cultural dimensions, the common challenges you will face, and the industry standards that ensure success.

You now understand that CI/CD is not merely a technical tooling problem but a **holistic approach to software delivery** that requires cultural commitment, architectural discipline, and continuous improvement.

In **Chapter 2: Essential Concepts and Terminology**, we will build upon this foundation by diving deep into the building blocks of modern CI/CD systems. You will learn about version control strategies, the distinctions between build, deploy, and release, the concept of immutable infrastructure, and the philosophy of GitOps. We will demystify terms like "Pipeline Stages," "Gates," and "Artifacts," ensuring you have the precise vocabulary needed before we begin working with Docker containers in Chapter 4. Understanding these concepts is crucial because they form the language we will use throughout the remainder of this handbook when implementing practical CI/CD solutions with Docker and Kubernetes.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <span style='color:gray; font-size:1.05em;'>Previous</span>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../11. real_world_projects/1. simple_web_app.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
